## Learning ipywidgets: a very simple d3 "CircleWidget" with a js CircleView 


[github conversation](https://github.com/ipython/ipywidgets/issues/838#issuecomment-255506936)|

In [122]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    },
});

<IPython.core.display.Javascript object>

In [123]:
import ipywidgets as widgets
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate

class CircleWidget(widgets.DOMWidget):
    _view_name = Unicode('CircleView').tag(sync=True)
    _view_module = Unicode('circle').tag(sync=True)
    radius = Int(100).tag(sync=True)
    circles = Tuple().tag(sync=True)
    def drawCircle(self, x, y, radius, fillColor="white", borderColor="black"):
       newCircle = {"x": x,  "y": y, "radius": radius, "fillColor": fillColor, "borderColor": borderColor}
       self.circles = (newCircle,) + self.circles


In [124]:
%%javascript
"use strict";

require.undef('circle');

define('circle', ["jupyter-js-widgets", "d3"], function(widgets, d3) {
    
    var CircleView = widgets.DOMWidgetView.extend({

        initialize: function() {
           console.log("---- initialize, this:");
           console.log(this);
           this.circles = [];
           },

        createDiv: function(){
            var div = $("<div id='d3DemoDiv' style='border:1px solid red; height: 300px; width: 600px'>")
            return(div);
            },
 
        createCanvas: function(){
           var svg = d3.select("#d3DemoDiv")
                       .append("svg")
                       .attr("id", "svg").attr("width", 600).attr("height", 300);
           this.svg = svg;
           console.log("---- createCanvas notion of this:");
           console.log(this);
           var circleView = this;
           svg.on('click', function() {
              var coords = d3.mouse(this);
              console.log(coords);
              //debugger;
              var newCircle = {x: coords[0], y: coords[1], radius: 20,
                               borderColor: "black", fillColor: "beige"};
              circleView.circles.push(newCircle);
              circleView.drawCircle(newCircle);
              //debugger;
              circleView.model.set("circles", JSON.stringify(circleView.circles));
              circleView.touch();
              });
           }, 

        drawCircle: function(obj){
           console.log("---- drawCircle notion of this:");
           console.log(this);
           this.svg.append("circle")
              .style("stroke", "gray")
              .style("fill", "white")
              .attr("r", obj.radius)
              .attr("cx", obj.x)
              .attr("cy", obj.y)
              .on("mouseover", function(){d3.select(this).style("fill", "aliceblue");})
              .on("mouseout",  function(){d3.select(this).style("fill", "white");});
            },

        render: function() { 
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:circles', this._circles_changed, this);
            console.log("---- render's notion of this:");
            console.log(this);
            var circleView = this;
            function myFunc(){
               circleView.createCanvas()
               }
            setTimeout(myFunc, 500);
            },

        _circles_changed: function() {
           console.log("_circles_changed")
           var newCircle = this.model.get("circles")[0];
           console.log("viewCircle.circles before adding new from python")
           console.log(this.circles);
           this.circles.push(newCircle);
           console.log(this.model.get('circles'));
           this.drawCircle(newCircle);
           }

    });
    return {
        CircleView : CircleView
    };
});

<IPython.core.display.Javascript object>

In [125]:
cw = CircleWidget(width=500, height=300)
cw

In [126]:
cw.drawCircle(x=30, y=30, radius=10)

In [127]:
cw.drawCircle(x=400, y=200, radius=30)

In [128]:
cw.circles

({'borderColor': 'black',
  'fillColor': 'white',
  'radius': 30,
  'x': 400,
  'y': 200},
 {'borderColor': 'black',
  'fillColor': 'white',
  'radius': 10,
  'x': 30,
  'y': 30})

In [121]:
cw.circles


()